<a href="https://colab.research.google.com/github/codeguima/data-science/blob/jhonnycodedev/Lista%20NPS%20Positivo_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORTANDO A LISTA PARA ANALISE, AVALIAÇÃO FINAL DATA SCIENCE

In [1]:
import pandas as pd


In [2]:
from google.colab import files
uploaded = files.upload()


Saving Lista NPS Positivo_V4 (1).xlsx to Lista NPS Positivo_V4 (1).xlsx


In [5]:
import io

# Substitua 'nome_do_arquivo.xlsx' pelo nome do seu arquivo
df = pd.read_excel(io.BytesIO(uploaded['Lista NPS Positivo_V4 (1).xlsx']), engine='openpyxl')


#**ITEM 1 DA LISTA**


In [6]:
# Função para classificar as notas
df['target'] = df['nota'].apply(lambda x: 'Promotor' if x >= 9 else ('Neutro' if x >= 7 else 'Detrator'))


In [ ]:
#print(df.groupby('target').count())

#**ITEMS 2, 3, 4 ,5 E 6 DA LISTA**

In [26]:

# Filtrar para "Grupo de Produto" == "Grupo 2"
grupo_2_df = df[(df['Grupo de Produto'] == 'Grupo 2') & (df['mercado'] == 'BRASIL')]


# Definir as regiões com base nos estados
regioes = {
    "Norte": ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
    "Nordeste": ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    "Centro-Oeste": ['DF', 'GO', 'MT', 'MS'],
    "Sul": ['PR', 'RS', 'SC'],
    "Suldeste": ['ES', 'MG', 'RJ', 'SP']
}

# Função para definir a região com base no estado
def obter_regiao(estado):
    for regiao, estados in regioes.items():
        if estado in estados:
            return regiao
    return 'Outros(não identificado)'  # Para estados fora das regiões definidas



In [27]:
# Crie a coluna 'regiao' se ainda não tiver feito
grupo_2_df['regiao'] = grupo_2_df['estado'].apply(obter_regiao)

# Crie um dicionário para armazenar os DataFrames
dfs_por_regiao = {}

# Use o groupby para dividir o DataFrame por 'regiao'
for regiao, df_regiao in grupo_2_df.groupby('regiao'):
    dfs_por_regiao[regiao] = df_regiao


<ipython-input-27-40cb2707167c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_2_df['regiao'] = grupo_2_df['estado'].apply(obter_regiao)


In [28]:
df_norte = dfs_por_regiao['Norte']
df_sul = dfs_por_regiao['Sul']
df_centro_oeste = dfs_por_regiao['Centro-Oeste']
df_nordeste = dfs_por_regiao['Nordeste']
df_sudeste = dfs_por_regiao['Suldeste']


**TESTE ITEM 7**

In [38]:
# Garantir que a coluna 'data_resposta' está no formato datetime no df_norte
df_norte['data_resposta'] = pd.to_datetime(df_norte['data_resposta'], dayfirst=True)

# Adicionar coluna 'safra' (apenas o ano da data) no df_norte
df_filter = df_norte['safra'] = df_norte['data_resposta'].dt.year



In [60]:
# Volumetria total por safra
df_dg = volumetria_safra = df_norte.groupby('safra').size().reset_index(name='quantidade')
volumetria_safra['percentual'] = volumetria_safra['quantidade'] / volumetria_safra['quantidade'].sum() * 100




In [55]:
# Volumetria por período de pesquisa e safra
volumetria_periodo = df_norte.groupby(['Periodo de Pesquisa', 'safra']).size().reset_index(name='quantidade')

In [57]:
# Calcular o percentual com transform
volumetria_periodo['percentual'] = volumetria_periodo.groupby('Periodo de Pesquisa')['quantidade'].transform(lambda x: x / x.sum() * 100)


In [58]:

# Soma total de safras (verificação)
total_safras = volumetria_safra['quantidade'].sum()
verificacao_total = len(df_norte) == total_safras

In [59]:
# Resultados
print("Volumetria por Safra (Total):\n", volumetria_safra)
print("\nVolumetria por Período de Pesquisa:\n", volumetria_periodo)
print(f"\nVerificação Total: {'Correto' if verificacao_total else 'Incorreto'}")

Volumetria por Safra (Total):
    safra  quantidade  percentual
0   2022          25   14.880952
1   2023          81   48.214286
2   2024          62   36.904762

Volumetria por Período de Pesquisa:
    Periodo de Pesquisa  safra  quantidade  percentual
0            12 a 18 M   2022           5   12.820513
1            12 a 18 M   2023          20   51.282051
2            12 a 18 M   2024          14   35.897436
3            18 a 30 M   2022           1    2.702703
4            18 a 30 M   2023          29   78.378378
5            18 a 30 M   2024           7   18.918919
6              3 a 6 M   2022           9   18.367347
7              3 a 6 M   2023          16   32.653061
8              3 a 6 M   2024          24   48.979592
9             6 a 12 M   2022          10   23.255814
10            6 a 12 M   2023          16   37.209302
11            6 a 12 M   2024          17   39.534884

Verificação Total: Correto


#CÓDIGO FINAL ITEM 7


In [61]:
#ITEM 7 DA LISTA

# Função para classificar as notas
df_norte['target'] = df_norte['nota'].apply(lambda x: 'Promotor' if x >= 9 else ('Neutro' if x >= 7 else 'Detrator'))

# Verifique se 'safra' foi criada antes, se não, crie-a:
df_norte['data_resposta'] = pd.to_datetime(df_norte['data_resposta'], dayfirst=True)
df_norte['safra'] = df_norte['data_resposta'].dt.year

# Calcular volumetria por safra e target
volumetria_safra_target = df_norte.groupby(['safra', 'target']).size().reset_index(name='quantidade')

# Calcular percentuais dentro de cada safra
volumetria_safra_target['percentual'] = volumetria_safra_target.groupby('safra')['quantidade'].transform(lambda x: x / x.sum() * 100)

# Resultado
print(volumetria_safra_target)


   safra    target  quantidade  percentual
0   2022  Detrator           2    8.000000
1   2022    Neutro           5   20.000000
2   2022  Promotor          18   72.000000
3   2023  Detrator           7    8.641975
4   2023    Neutro          16   19.753086
5   2023  Promotor          58   71.604938
6   2024  Detrator           7   11.290323
7   2024    Neutro           7   11.290323
8   2024  Promotor          48   77.419355


#ITEM 9 DA LISTA

correlação de spearman, entre a variável nota e as demais variáveis de perguntas

In [63]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import seaborn as sns
import matplotlib.pyplot as plt

# Supondo que 'df' seja o seu DataFrame já filtrado para o grupo de produto e para o mercado BRASIL

# Defina as colunas de perguntas com notas (de 0 a 10) em uma lista
perguntas = ['capacidade operacional (hectares por hora) (csat)',
             'adequação as diversas operações e implementos (csat)',
             'facilidade de operação (csat)',
             'conforto e ergonomia (csat)',
             'disponibilidade e confiabilidade mecânica  (csat)',
             'facilidade para realização de manutenções (csat)',
             'custo de manutenção (csat)',
             'consumo de combustível (litros por hectares) (csat)',
             'adaptabilidade as mais diversas condições de trabalho (csat)',
             'facilidade de uso do piloto automático (csat)',
             'geração e transmissão de dados para gestão da frota (csat)',
             'geração e transmissão de dados para gestão agrícola (csat)',
             'como você avalia a qualidade e a confiabilidade do seu <modelo> (considere o acabamento e a aparência da sua máquina, falhas, problemas de confiabilidade ou avarias que você possa ter tido)? (csat)',
             'agora, considere características específicas do seu <modelo>, como você avalia seu desempenho (csat)',
             'como você avalia o conforto e a ergonomia do seu <modelo>  (considere, por exemplo, o conforto do assento, a visibilidade da cabine, a compreensão e o layout dos controles)? (csat)',
             '¿está satisfecho con la calidad del producto? (csat)']  # Substitua com os nomes reais das suas colunas de perguntas

# Calcule a correlação de Spearman entre a 'nota' e as variáveis de perguntas
def calcular_correlacao_spearman(df, perguntas, target_col='nota'):
    # Calcula a correlação de Spearman para cada variável de pergunta
    correlacoes = {}
    for pergunta in perguntas:
        corr, _ = spearmanr(df[target_col], df[pergunta], nan_policy='omit')
        correlacoes[pergunta] = corr
    return correlacoes

# Calcular as correlações para o grupo inteiro
correlacoes_grupo = calcular_correlacao_spearman(df, perguntas)

# Ordenar a lista de correlações do maior para o menor
correlacoes_grupo_sorted = sorted(correlacoes_grupo.items(), key=lambda x: abs(x[1]), reverse=True)
correlacoes_grupo_sorted_df = pd.DataFrame(correlacoes_grupo_sorted, columns=['Pergunta', 'Correlação'])

# Exibir as correlações ordenadas
print(correlacoes_grupo_sorted_df)


                                             Pergunta  Correlação
0   ¿está satisfecho con la calidad del producto? ...    0.692549
1   como você avalia a qualidade e a confiabilidad...    0.675211
2   disponibilidade e confiabilidade mecânica  (csat)    0.658972
3   adequação as diversas operações e implementos ...    0.634115
4   adaptabilidade as mais diversas condições de t...    0.627401
5   capacidade operacional (hectares por hora) (csat)    0.624406
6   agora, considere características específicas d...    0.613134
7                       facilidade de operação (csat)    0.608312
8   geração e transmissão de dados para gestão agr...    0.604890
9    facilidade para realização de manutenções (csat)    0.600659
10  geração e transmissão de dados para gestão da ...    0.599419
11                        conforto e ergonomia (csat)    0.554529
12      facilidade de uso do piloto automático (csat)    0.535848
13                         custo de manutenção (csat)    0.477408
14  consum

In [64]:
# Visualizar as Correlações com Formatação (Cores)
# Agora, podemos destacar as correlações fortes, médias e fracas usando cores:

#Correlação forte: acima de 0.7 ou abaixo de -0.7 (vermelho)
#Correlação média: entre 0.4 e 0.7 ou entre -0.4 e -0.7 (azul)
#Correlação fraca: abaixo de 0.4 ou acima de -0.4 (verde)

# Função para aplicar as cores conforme a correlação
def colorir_correlações(val):
    if abs(val) >= 0.7:
        color = 'red'  # Forte
    elif abs(val) >= 0.4:
        color = 'blue'  # Médio
    else:
        color = 'green'  # Fraco
    return f'background-color: {color}'

# Criar um DataFrame para aplicar o estilo
correlacoes_grupo_sorted_df = correlacoes_grupo_sorted_df.set_index('Pergunta')
styled_df = correlacoes_grupo_sorted_df.style.applymap(lambda x: colorir_correlações(x), subset=['Correlação'])

# Exibir a tabela estilizada
styled_df


<ipython-input-64-b92f904ffccc>:20: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = correlacoes_grupo_sorted_df.style.applymap(lambda x: colorir_correlações(x), subset=['Correlação'])


,Correlação
Pergunta,
¿está satisfecho con la calidad del producto? (csat),0.692549
"como você avalia a qualidade e a confiabilidade do seu (considere o acabamento e a aparência da sua máquina, falhas, problemas de confiabilidade ou avarias que você possa ter tido)? (csat)",0.675211
disponibilidade e confiabilidade mecânica (csat),0.658972
adequação as diversas operações e implementos (csat),0.634115
adaptabilidade as mais diversas condições de trabalho (csat),0.627401
capacidade operacional (hectares por hora) (csat),0.624406
"agora, considere características específicas do seu , como você avalia seu desempenho (csat)",0.613134
facilidade de operação (csat),0.608312
geração e transmissão de dados para gestão agrícola (csat),0.604890


**Calculando as Correlações para Cada Região:**

In [82]:
# Exemplo para calcular as correlações por região

# Dicionário para armazenar as correlações por região
correlacoes_por_regiao = {}

# Calcular as correlações por região
for regiao, df_regiao in {'Norte': df_norte, 'Sul': df_sul, 'Centro-Oeste': df_centro_oeste, 'Nordeste': df_nordeste, 'Sudeste': df_sudeste}.items():
    correlacoes_por_regiao[regiao] = calcular_correlacao_spearman(df_regiao, perguntas)

# Exibir as correlações por região
for regiao, correlacoes in correlacoes_por_regiao.items():
    print(f"Correlação para a região: {regiao}")
    # Ordena as correlações pela magnitude da correlação (do maior para o menor valor absoluto)
    correlacoes_sorted = sorted(correlacoes.items(), key=lambda x: abs(x[1]), reverse=True)
    print(pd.DataFrame(correlacoes_sorted, columns=['Pergunta', 'Correlação']))
    print("\n")



Correlação para a região: Norte
                                             Pergunta  Correlação
0   ¿está satisfecho con la calidad del producto? ...    0.814803
1   disponibilidade e confiabilidade mecânica  (csat)    0.721723
2   como você avalia a qualidade e a confiabilidad...    0.673533
3   adequação as diversas operações e implementos ...    0.670797
4   adaptabilidade as mais diversas condições de t...    0.664106
5    facilidade para realização de manutenções (csat)    0.644083
6                       facilidade de operação (csat)    0.624613
7   geração e transmissão de dados para gestão agr...    0.617817
8   capacidade operacional (hectares por hora) (csat)    0.611271
9   agora, considere características específicas d...    0.608413
10  geração e transmissão de dados para gestão da ...    0.597596
11                        conforto e ergonomia (csat)    0.583408
12                         custo de manutenção (csat)    0.524849
13      facilidade de uso do piloto automáti

In [83]:
# Função para aplicar a cor nas correlações
def colorir_correlações(val):
    if abs(val) >= 0.7:
        color = 'red'  # Forte
    elif abs(val) >= 0.4:
        color = 'blue'  # Médio
    else:
        color = 'green'  # Fraco
    return f'background-color: {color}'

# Visualizando as correlações por região com cores
for regiao, correlacoes in correlacoes_por_regiao.items():
    print(f"Correlação para a região: {regiao}")
    correlacoes_sorted = sorted(correlacoes.items(), key=lambda x: abs(x[1]), reverse=True)
    correlacoes_df = pd.DataFrame(correlacoes_sorted, columns=['Pergunta', 'Correlação'])

    # Aplicando a coloração nas correlações
    styled_df = correlacoes_df.style.applymap(lambda x: colorir_correlações(x), subset=['Correlação'])

    # Exibindo a tabela estilizada
    display(styled_df)  # Exibe a tabela no Jupyter ou notebook
    print("\n")


Correlação para a região: Norte


<ipython-input-83-691b2ce3f226>:18: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = correlacoes_df.style.applymap(lambda x: colorir_correlações(x), subset=['Correlação'])


,Pergunta,Correlação
0,¿está satisfecho con la calidad del producto? (csat),0.814803
1,disponibilidade e confiabilidade mecânica (csat),0.721723
2,"como você avalia a qualidade e a confiabilidade do seu (considere o acabamento e a aparência da sua máquina, falhas, problemas de confiabilidade ou avarias que você possa ter tido)? (csat)",0.673533
3,adequação as diversas operações e implementos (csat),0.670797
4,adaptabilidade as mais diversas condições de trabalho (csat),0.664106
5,facilidade para realização de manutenções (csat),0.644083
6,facilidade de operação (csat),0.624613
7,geração e transmissão de dados para gestão agrícola (csat),0.617817
8,capacidade operacional (hectares por hora) (csat),0.611271
9,"agora, considere características específicas do seu , como você avalia seu desempenho (csat)",0.608413




Correlação para a região: Sul


<ipython-input-83-691b2ce3f226>:18: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = correlacoes_df.style.applymap(lambda x: colorir_correlações(x), subset=['Correlação'])


...

<ipython-input-83-691b2ce3f226>:18: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = correlacoes_df.style.applymap(lambda x: colorir_correlações(x), subset=['Correlação'])


,Pergunta,Correlação
0,adequação as diversas operações e implementos (csat),0.739814
1,facilidade para realização de manutenções (csat),0.698253
2,adaptabilidade as mais diversas condições de trabalho (csat),0.680515
3,capacidade operacional (hectares por hora) (csat),0.679852
4,disponibilidade e confiabilidade mecânica (csat),0.669049
5,facilidade de operação (csat),0.660049
6,facilidade de uso do piloto automático (csat),0.623286
7,conforto e ergonomia (csat),0.612225
8,"como você avalia a qualidade e a confiabilidade do seu (considere o acabamento e a aparência da sua máquina, falhas, problemas de confiabilidade ou avarias que você possa ter tido)? (csat)",0.598307
9,custo de manutenção (csat),0.597270




Correlação para a região: Sudeste


<ipython-input-83-691b2ce3f226>:18: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = correlacoes_df.style.applymap(lambda x: colorir_correlações(x), subset=['Correlação'])


,Pergunta,Correlação
0,disponibilidade e confiabilidade mecânica (csat),0.711304
1,capacidade operacional (hectares por hora) (csat),0.704048
2,¿está satisfecho con la calidad del producto? (csat),0.702845
3,facilidade de uso do piloto automático (csat),0.700078
4,geração e transmissão de dados para gestão da frota (csat),0.698731
5,geração e transmissão de dados para gestão agrícola (csat),0.679432
6,"como você avalia a qualidade e a confiabilidade do seu (considere o acabamento e a aparência da sua máquina, falhas, problemas de confiabilidade ou avarias que você possa ter tido)? (csat)",0.662960
7,facilidade de operação (csat),0.655804
8,adequação as diversas operações e implementos (csat),0.655113
9,adaptabilidade as mais diversas condições de trabalho (csat),0.640264


-->

Apresentar também a lista de correlações Safrada, uma lista por safra, replicando o trabalho para cada região e período de pesquisa.

In [84]:
# Função para calcular a correlação de Spearman para cada safra
def calcular_correlacao_safra(df, perguntas, target_col='nota'):
    correlacoes = {}
    for pergunta in perguntas:
        corr, _ = spearmanr(df[target_col], df[pergunta], nan_policy='omit')
        correlacoes[pergunta] = corr
    return correlacoes


Verificando a correlação de Spearman entre quantidade e percentual

In [100]:
from scipy.stats import spearmanr

# Função para calcular correlação de Spearman
def calcular_correlacao_safra_target(df, col1, col2):
    corr, _ = spearmanr(df[col1], df[col2], nan_policy='omit')
    return corr

# Dicionário para armazenar as correlações por safra
correlacoes_por_safra_target = {}

# Calculando a correlação entre 'quantidade' e 'percentual' por safra
for safra, df_safra in volumetria_safra_target.groupby('safra'):
    correlacoes_por_safra_target[safra] = calcular_correlacao_safra_target(df_safra, 'quantidade', 'percentual')

# Exibir as correlações por safra
for safra, correlacao in correlacoes_por_safra_target.items():
    print(f"Correlação entre 'quantidade' e 'percentual' para a Safra: {safra} = {correlacao}")


Correlação entre 'quantidade' e 'percentual' para a Safra: 2022 = 1.0
Correlação entre 'quantidade' e 'percentual' para a Safra: 2023 = 1.0
Correlação entre 'quantidade' e 'percentual' para a Safra: 2024 = 1.0


In [101]:
# Função para aplicar a cor nas correlações
def colorir_correlações(val):
    if abs(val) >= 0.7:
        color = 'red'  # Correlação forte
    elif abs(val) >= 0.4:
        color = 'blue'  # Correlação média
    else:
        color = 'green'  # Correlação fraca
    return f'background-color: {color}'

# Visualizando as correlações com cores
correlacoes_df = pd.DataFrame(list(correlacoes_por_safra_target.items()), columns=['Safra', 'Correlação'])

# Aplicando a coloração nas correlações
styled_df = correlacoes_df.style.applymap(lambda x: colorir_correlações(x), subset=['Correlação'])

# Exibindo a tabela estilizada
display(styled_df)


<ipython-input-101-3780b295463c>:15: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = correlacoes_df.style.applymap(lambda x: colorir_correlações(x), subset=['Correlação'])


,Safra,Correlação
0,2022,1.000000
1,2023,1.000000
2,2024,1.000000


In [102]:
# Função para calcular a correlação de Spearman para 'target' e outras variáveis
def calcular_correlacao_por_target(df, target, col):
    df_target = df[df['target'] == target]
    corr, _ = spearmanr(df_target[col], df_target['quantidade'], nan_policy='omit')
    return corr

# Dicionário para armazenar as correlações por target
correlacoes_por_target = {}

# Calcular a correlação entre 'quantidade' e 'target'
for target in volumetria_safra_target['target'].unique():
    correlacoes_por_target[target] = calcular_correlacao_por_target(volumetria_safra_target, target, 'quantidade')

# Exibir as correlações
print("Correlação entre 'quantidade' e 'target':")
for target, corr in correlacoes_por_target.items():
    print(f"Target {target}: {corr}")


Correlação entre 'quantidade' e 'target':
Target Detrator: 1.0
Target Neutro: 1.0
Target Promotor: 1.0
